# Simulations

**Simulations** are a description of computations that provide useful information about the possible behaviors of the system
being modeled. These are great tools for makings guesses about the likelihood of future outcomes.


Although simulations can never perfectly model reality, they give us a spectrum of possible outcomes that could manifest in reality. As we saw in the dice rolling simulations, with an accurate simulation and large enough trial set, we can begin to approximate the likelihood of given outcomes actually happening. 

It's important to remember that simulations are only an approximation to reality. As George Box said, "All models are wrong, but some are useful." Furthermore, models should be regarded descriptive, not prescriptive. They can't tell you what to do. they just provide you the chances of what might happen. You must decide your risk aversion levels and make decisions on your own. Simulations are used to model:

- systems that are mathematically intractible 
    - If you roll dice for a 6, there is a 1/6 chance of getting a 6. As you roll dice, you continuously get closer to rolling a 6 every 1 in 6 times. 
    - This is known. Sometimes the chances of events happening are not known and not so simple. 
- to extract useful intermediate results
     - we should buy this stock because it's probably going to increase in price soon.    
- good for development by successive refinement and 'what if' questions
    - That is to say you can start of with a simple/bad model of reality and refine it iteratively
    
In the following sections, we will introduce simulations with random walks.

## Random Walks

![random walk graph](https://i.stack.imgur.com/rUI8q.png)

Random walks are super useful in modeling the stalk market and other diffusion processes (like chemicals or populations spreading around the world). They are not too complicated to simulate. They provide a great chance to continue building skills in classes and plotting.

<a href="https://www.youtube.com/watch?v=6wUD_gp5WeE
" target="_blank"><img src="http://img.youtube.com/vi/YOUTUBE_VIDEO_ID_HERE/0.jpg" 
alt="IMAGE ALT TEXT HERE" width="240" height="180" border="10" /></a>

# The Drunkard's Walk

In the folling lines, we will create a simulation of a drunk guy walking through a field. Imagine somebody so drunk, they are just walking in every which direction. They have an equal chance of going North, South, East or West. Then we will imagine a drunkard that has enough awareness to move slowly in some cardinal direction. We're also going to create a field with wormholes in it that teleport the drunks... just for fun. and to see how it affects the results.

## Random Walk Classes

There will be three base classes needed to implement this random walk. Later, we will create subclasses for field and Drunk.

- `Location` The coordinates within the field that a given drunkard is at.

- `Field` The actual area the drunks can walk.

- `Drunk` The person walking around at random.

In [33]:
import random

#### Location Class

When a `Location` object is instantiated, `x` and `y` coordinates are defined.

The `.move()` method takes in `deltaX` and `deltaY`, which are equal to the change from current `x` and `y` values. It returns a recursive call and updates location by summing current location with change in location with `self.x + deltaX` and `self.y + deltaY`.

The `.getX()` and `.getY()` getter methods safely return the current values of `x` and `y`, respectively.


The `distFrom(self, other)` method returns the distance between a location and another location passed in as an argument, by using the pythagorean theorem.


`__str__(self)` specifies the string representation for `Location`

In [21]:
class Location(object):
    def __init__(self, x, y):
        """x and y are numbers"""
        self.x = x
        self.y = y

    def move(self, deltaX, deltaY):
        """deltaX and deltaY are numbers"""
        return Location(self.x + deltaX, self.y + deltaY)

    def getX(self):
        return self.x

    def getY(self):
        return self.y

    def distFrom(self, other):
        ox = other.x
        oy = other.y
        xDist = self.x - ox
        yDist = self.y - oy
        return (xDist**2 + yDist**2)**0.5

    def __str__(self):
        return '<' + str(self.x) + ', ' + str(self.y) + '>'

#### Field Class

`Field` object's constructor method creates attribute `self.drunks` and sets it to equal an empty dicitonary.

The `addDrunk` method takes in two parameters: `drunk` (a `drunk` object) and `loc` (a `Location` object associated with that `drunk`). It checks to makes sure the `drunk` isn't in the `Field` object's `self.drunk` dictionary yet, and if not, adds it in as a key with the value set to `loc`.

The `moveDrunk` object takes in `drunk` as a parameter. If the `drunk` passed in is not in the `self.drunks` dictionary, a value error is raised. The value of `drunk.takesStep()` is unpacked into variables `xDist` and `yDist`. (For `usualDrunk` objects, the `drunk.takesStep()` method makes the `drunk` take a single, ranom step in any direction of the field.) `currentLocation` is set to equal the `loc` object for the `drunk`, and then the dictionary value for the passed in `drunk` is reset to be the value of `currentLocation.move(xDist, yDist)`. (The `.move()` method just move

The `getLoc(self, drunk)` returns the location of a `drunk` or else raises a `ValueError` if drunk is not in `Field`.

In [22]:
class Field(object):
    def __init__(self):
        self.drunks = {}
        
    def addDrunk(self, drunk, loc):
        if drunk in self.drunks:
            raise ValueError('Duplicate drunk')
        else:
            self.drunks[drunk] = loc
            
    def moveDrunk(self, drunk):
        if drunk not in self.drunks:
            raise ValueError('Drunk not in field')
        xDist, yDist = drunk.takeStep()
        currentLocation = self.drunks[drunk]
        #use move method of Location to get new location
        self.drunks[drunk] = currentLocation.move(xDist, yDist)
        
    def getLoc(self, drunk):
        if drunk not in self.drunks:
            raise ValueError('Drunk not in field')
        return self.drunks[drunk]

#### Drunk

This class is meant to be a base class.. That means it never gets used on its own. Subclasses inherit it, and those are what actually get used. 

On initialization, the `name` parameter is set to `None`, but can be manually overwritten. 

the `__str__()` method returns `Anonymous` if `self.name == None`, else returns `self.name` 

In [23]:
class Drunk(object):
    def __init__(self, name = None):
        """Assumes name is a str"""
        self.name = name

    def __str__(self):
        if self != None:
            return self.name
        return 'Anonymous'

This `drunk` subclass simply takes a random step in any direction.

In [24]:
class UsualDrunk(Drunk):
    def takeStep(self):
        stepChoices = [(0,1), (0,-1), (1, 0), (-1, 0)]
        return random.choice(stepChoices)

The `ColdDrunk` takes a random step in any direction, but the movement is slightly weighted southward.

In [25]:
class ColdDrunk(Drunk):
    def takeStep(self):
        stepChoices = [(0.0,0.9), (0.0,-1.1),
                       (1.0, 0.0), (-1.0, 0.0)]
        return random.choice(stepChoices)

## Random Walk Functions

The following functions implement our random walk. 

`walk` saves the starting location of a drunk `d` in a field `f` into variable `start`. 
The function then iterates through a range the length of `numSteps` and moves the drunk around the field. Finally it returns the distance between `start` to the final distance after walking `numSteps` times.

In [26]:
def walk(f, d, numSteps):
    """Assumes: f a Field, d a Drunk in f, and numSteps an int >= 0.
       Moves d numSteps times, and returns the distance between
       the final location and the location at the start of the 
       walk."""
    start = f.getLoc(d)
    for s in range(numSteps):
        f.moveDrunk(d)
    return start.distFrom(f.getLoc(d))

This takes in 3 parameters `numSteps`, `numTrials` and `dClass`. It returns a list `distances` that contains `numTrials` number of walks each `numSteps` in length, using drunks of `dClass`.

In [27]:
def simWalks(numSteps, numTrials, dClass):
    """Assumes numSteps an int >= 0, numTrials an int > 0,
         dClass a subclass of Drunk
       Simulates numTrials walks of numSteps steps each.
       Returns a list of the final distances for each trial"""
    Homer = dClass()
    origin = Location(0, 0)
    distances = []
    for t in range(numTrials):
        f = Field()
        f.addDrunk(Homer, origin)
        distances.append(round(walk(f, Homer, numSteps), 1))
    return distances


`drunkTest` takes in 3 parameters: `walkLengths` (a tuple of integers, representing the `numSteps` in a walk), `numTrials` (which is essentially the number of drunks to takes the walk), and `dClass` (the class of drunk to take the walk).

The function then prints out stats with various walk lengths after running our random walk simulation code above.

In [34]:
def drunkTest(walkLengths, numTrials, dClass):
    """Assumes walkLengths a sequence of ints >= 0
         numTrials an int > 0, dClass a subclass of Drunk
       For each number of steps in walkLengths, runs simWalks with
         numTrials walks and prints results"""
    for numSteps in walkLengths:
        distances = simWalks(numSteps, numTrials, dClass)
        print(dClass.__name__, 'random walk of', numSteps, 'steps')
        print(' Mean =', round(sum(distances)/len(distances), 4))
        print(' Max =', max(distances), 'Min =', min(distances))

This code calls `drunkTest` with specificed parameters. As noted above, `random.seed(0)` isolates the randomization sequence so results can be validated and debugged.

In [35]:
random.seed(0)
drunkTest((10, 1000, 1000, 10000), 100, UsualDrunk)

UsualDrunk random walk of 10 steps
 Mean = 2.863
 Max = 7.2 Min = 0.0
UsualDrunk random walk of 1000 steps
 Mean = 26.812
 Max = 66.3 Min = 4.2
UsualDrunk random walk of 1000 steps
 Mean = 30.873
 Max = 71.8 Min = 1.4
UsualDrunk random walk of 10000 steps
 Mean = 88.991
 Max = 226.5 Min = 10.0


`simAll` runs `drunkTest` but lets us run it on multiple kinds of drunks.

In [36]:
def simAll(drunkKinds, walkLengths, numTrials):
    for dClass in drunkKinds:
        drunkTest(walkLengths, numTrials, dClass)

In [37]:
random.seed(0)
simAll((UsualDrunk, ColdDrunk),
       (1, 10, 100, 1000, 10000), 100)

UsualDrunk random walk of 1 steps
 Mean = 1.0
 Max = 1.0 Min = 1.0
UsualDrunk random walk of 10 steps
 Mean = 2.948
 Max = 7.2 Min = 0.0
UsualDrunk random walk of 100 steps
 Mean = 8.453
 Max = 21.6 Min = 1.4
UsualDrunk random walk of 1000 steps
 Mean = 27.138
 Max = 72.8 Min = 1.4
UsualDrunk random walk of 10000 steps
 Mean = 90.661
 Max = 222.9 Min = 5.1
ColdDrunk random walk of 1 steps
 Mean = 0.996
 Max = 1.1 Min = 0.9
ColdDrunk random walk of 10 steps
 Mean = 2.788
 Max = 7.1 Min = 0.2
ColdDrunk random walk of 100 steps
 Mean = 9.602
 Max = 22.9 Min = 0.4
ColdDrunk random walk of 1000 steps
 Mean = 53.525
 Max = 108.3 Min = 7.3
ColdDrunk random walk of 10000 steps
 Mean = 495.128
 Max = 654.6 Min = 323.9
